# Load and georeference vectorized results

In [3]:
import os
import json
from operator import contains

import geopandas as gp
import pandas as pd
from shapely import geometry as geom

# Indicate location to process
case = 'Hilversum'

if case == 'Apeldoorn':
    #paths for Apeldoorn set, kad_path contains parcel information from te cadastral map, json_path the vectorized results
    kad_path = r'C:\Werkmap\3D_appartementsrechten\Voorbeeld_data\Testcase Apeldoorn\percelen_apeldoorn.shp'
    json_path = r'C:\Werkmap\3D_appartementsrechten\training_data\apeldoorn_test_def\observations\snapshots\latest'
elif case == 'Hilversum':
    #paths for Hilversum set
    kad_path = r'C:\Users\NietLottede\Documents\Lotte\original_data\aanvullende_data\Hilversum\Percelen_aktes_Hilversum.shp'
json_path = r"C:\Users\NietLottede\Documents\Lotte\original_data\gevectoriseerde_set\hilversum_set\observations\snapshots\latest"
out_path = r"C:\Users\NietLottede\Documents\Lotte\original_data\gevectoriseerde_set\hilversum_set\output2"
pand_path = r"C:\Users\NietLottede\Documents\Lotte\github_code\thesis\werkmap\pand_point.shp"
pand_geom_path = r"C:\Users\NietLottede\Documents\Lotte\github_code\thesis\werkmap\10-506-654.gpkg"

if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('output directory created')

files = os.listdir(json_path)

# simple function to estimate whether a plan is horizontally or vertically positioned, based on the positioning, calculate a rough first 'scale' estimate.
def calcScale(bbxobj,pix):
    xlen = float((bbxobj.maxx-bbxobj.minx).iloc[0])
    ylen = float((bbxobj.maxy-bbxobj.miny).iloc[0])
    if xlen < ylen:
        return ylen/float(pix)
    else:
        return xlen/float(pix)

# make polygon geometries based on points    
def egetGeometry(plist,pdict):
    gl = []
    for p in plist:
        gl.append(pdict[p])
    return geom.Polygon(gl)

# add attribute info to rooms
def addValue(cat,clist,room):
    try:
        clist.append(data['text'][data['rooms'][room][cat]]['value'])
    except:
        clist.append('')

kad_list = []
# get all the 'Kadastrale aanduidingen' (Gemeente, Sectie, Perceelnr)
for f in files:
    if f.endswith('.json'):
        parts = f.split('.')
        kad_list.append(parts[0])

# read all relevant parcels
kadpercelen = gp.read_file(kad_path)
kadpandpoint = gp.read_file(pand_path)

kadperceel_pand = gp.sjoin(kadpercelen,kadpandpoint, how='left', predicate='contains')

for kp in kad_list:
    parts = kp.split('_')
    selection=kadperceel_pand[kadperceel_pand.KAD_GEM.eq(parts[1]) & kadperceel_pand.SECTIE.eq(parts[2]) & kadperceel_pand.PERCEELNUM.eq(int(parts[3]))]
    kadperceel_pand['perceelnummer'] = parts[3]
    bp = selection.centroid
    bbx = selection.bounds
    with open(os.path.join(json_path,f'{kp}.latest.json')) as f:
        data = json.load(f)
    pixapprox = max(data['meta']['frontDimensions'])/3
    scale = calcScale(bbx,pixapprox)
    #create pointdict
    pointDict = {}
    for pt in data['points'].keys():
        x,y = data['points'][pt]['position']
        pointDict[pt] = [x * scale + float(bp.x.iloc[0]),(float(data['meta']['frontDimensions'][1]) - y) * scale + float(bp.y.iloc[0])]
    #empty list to create the attributes properly
    roomIDs = []
    appartementsnummer = []
    ruimteomschrijving = []
    verdiepingsaanduiding = []
    geometry = []
    attachment = []
    #filter per verdieping
    for r in data['rooms'].keys():
        roomIDs.append(r)
        addValue('appartementsnummer',appartementsnummer,r)
        addValue('ruimteomschrijving',ruimteomschrijving,r)
        addValue('verdiepingaanduiding',verdiepingsaanduiding,r)
        attachment.append(data['rooms'][r]['attachment'])
        geometry.append(egetGeometry(data['rooms'][r]['points'],pointDict))
    
    gdf = gp.GeoDataFrame(data=zip(roomIDs,verdiepingsaanduiding,appartementsnummer,ruimteomschrijving,attachment), geometry=geometry,crs="EPSG:28992",columns=['room','verdieping','appartement','ruimte','attachment'])
    
    #write different levels to different layers of gpkg
    for lvl in gdf.verdieping.unique():
        tpf = gdf[gdf.verdieping.eq(lvl)]
        tpf.to_file(os.path.join(out_path,'{}{}{}.gpkg'.format(parts[1],parts[2],parts[3])),driver='GPKG', layer=lvl)
    #print('Finished processing {}{}{}'.format(parts[1],parts[2],parts[3]))

    #print(kadperceel_pand[['PERCEELNUM', 'identifica' ]])
    #print(kadperceel_pand.info())

    pand_geom = gp.read_file(pand_geom_path, layer='pand')
    kadperceel_pand['identifica'] =  kadperceel_pand['identifica'].apply(lambda x: 'NL.IMBAG.Pand.' + str(x) if not str(x).startswith('NL.IMBAG.Pand.') else str(x))
    kadperceel_pand_geom = kadperceel_pand.merge(pand_geom, how='left', left_on='identifica', right_on='identificatie')
    print(kadperceel_pand_geom.info())



    # geopackage creation
roomIDs = []
appartementsnummer = []
ruimteomschrijving = []
verdiepingsaanduiding = []
geometry = []
attachment = []

def getGeometry(plist,pdict):
    gl = []
    for p in plist:
        gl.append(pdict[p])
    return geom.Polygon(gl)

def addValue(cat,clist,room):
    try:
        clist.append(data['text'][data['rooms'][room][cat]]['value'])
    except:
        clist.append('')

# filter the correct information for each building level
for r in data['rooms'].keys():
    roomIDs.append(r)
    addValue('appartementsnummer',appartementsnummer,r)
    addValue('ruimteomschrijving',ruimteomschrijving,r)
    addValue('verdiepingaanduiding',verdiepingsaanduiding,r)
    attachment.append(data['rooms'][r]['attachment'])
    geometry.append(getGeometry(data['rooms'][r]['points'],pointDict))

vec_gdf = gp.GeoDataFrame(data=zip(roomIDs,verdiepingsaanduiding,appartementsnummer,ruimteomschrijving,attachment), geometry=geometry,crs="EPSG:28992",columns=['room','verdiepin','apprtment','ruimte','attachment'])

# write different levels to different layers of gpkg
for lvl in vec_gdf.verdiepin.unique():
    tpf = vec_gdf[gdf.verdiepin.eq(lvl)]
    tpf.to_file(os.path.join(os.path.split(kad_path)[0],'{}{}{}.gpkg'.format(parts[1],parts[2],parts[3])),driver='GPKG', layer=lvl)

combined_gdf = kadperceel_pand_geom.merge(vec_gdf, how='left',left_on='PERCEELNUM',right_on='perceelnummer')

perceeltest = kadperceel_pand_geom[kadperceel_pand_geom['PERCEELNUM'] == 555]
perceeltestsave = perceeltest[['PERCEELNUM', 'geometry_x']]
perceeltestsave2 = perceeltest[['identificatie', 'geometry_y']]
perceeltestsave3 = perceeltest[['identificatie', 'geometry']]
perceeltestsave.to_file(os.path.join(out_path,'kadperceel_pand.gpkg'), driver='GPKG')
perceeltestsave2.to_file(os.path.join(out_path,'kadperceel_pand2.gpkg'), driver='GPKG')
perceeltestsave3.to_file(os.path.join(out_path,'kadperceel_pand3.gpkg'), driver='GPKG')


# parcel_gdf = combined_gdf[['PERCEELNUM', 'geometry_x']].rename(columns={'geometry_x': 'geometry'})
# parcel_gdf = gpd.GeoDataFrame(parcel_gdf, geometry='geometry', crs=combined_gdf.crs)
# parcel_gdf.to_file('output.gpkg', driver='GPKG', layer='parcel', index=False)
#
# pand_gdf = combined_gdf[['identificatie', 'geometry_y']].rename(columns={'geometry_y': 'geometry'})
# pand_gdf = gpd.GeoDataFrame(pand_gdf, geometry='geometry', crs=combined_gdf.crs)
# pand_gdf.to_file('output.gpkg', driver='GPKG', layer='pand', index=False)
#
# vec_gdf.to_file('output.gpkg', driver='GPKG', layer='rooms', index=False)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 76 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   OBJECT_ID                    408 non-null    object  
 1   GEMEENTE                     408 non-null    object  
 2   KAD_GEM                      408 non-null    object  
 3   SECTIE                       408 non-null    object  
 4   PERCEELNUM                   408 non-null    int64   
 5   DEEL                         408 non-null    object  
 6   NUMMER                       408 non-null    object  
 7   REEKS_CODE                   266 non-null    object  
 8   STUK_KIK                     408 non-null    object  
 9   SNAPSHOT_R                   408 non-null    object  
 10  geometry_x                   408 non-null    geometry
 11  index_right                  313 non-null    float64 
 12  gml_id                       313 non-null    object  
 1

AttributeError: 'GeoDataFrame' object has no attribute 'verdiepin'

In [18]:
#test code for generating approximate points
df = pd.DataFrame.from_dict(data=pointDict,orient='index',columns=['x','y'])
df['name'] = df.index
gdf = gp.GeoDataFrame(df, geometry=gp.points_from_xy(df.x, df.y))
gdf.to_file(os.path.join(os.path.split(kad_path)[0],'testPoints.shp'), driver='ESRI Shapefile')

C:\Users\NietLottede\PycharmProjects\PythonProject\.venv\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [20]:
# geopackage creation
roomIDs = []
appartementsnummer = []
ruimteomschrijving = []
verdiepingsaanduiding = []
geometry = []
attachment = []

def getGeometry(plist,pdict):
    gl = []
    for p in plist:
        gl.append(pdict[p])
    return geom.Polygon(gl)

def addValue(cat,clist,room):
    try:
        clist.append(data['text'][data['rooms'][room][cat]]['value'])
    except:
        clist.append('')
        
# filter the correct information for each building level
for r in data['rooms'].keys():
    roomIDs.append(r)
    addValue('appartementsnummer',appartementsnummer,r)
    addValue('ruimteomschrijving',ruimteomschrijving,r)
    addValue('verdiepingaanduiding',verdiepingsaanduiding,r)
    attachment.append(data['rooms'][r]['attachment'])
    geometry.append(getGeometry(data['rooms'][r]['points'],pointDict))
    
gdf = gp.GeoDataFrame(data=zip(roomIDs,verdiepingsaanduiding,appartementsnummer,ruimteomschrijving,attachment, kadastraleaanduiding), geometry=geometry,crs="EPSG:28992",columns=['room','verdiepin','apprtment','ruimte','attachment', 'kadastraleaanduiding'])

# write different levels to different layers of gpkg
for lvl in gdf.verdiepin.unique():
    tpf = gdf[gdf.verdiepin.eq(lvl)]
    tpf.to_file(os.path.join(os.path.split(kad_path)[0],'{}{}{}.gpkg'.format(parts[1],parts[2],parts[3])),driver='GPKG', layer=lvl)




,room,verdiepin,apprtment,ruimte,attachment,geometry
6,r15,begane grond schaal 1:100,7,,front,"POLYGON ((140158.889 470324.972, 140153.841 47..."


In [19]:
import fiona
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [8]:
data['rooms']['r1']

{'appartementsnummer': 't8',
 'attachment': 'front',
 'points': ['p11',
  'p33',
  'p394',
  'p45',
  'p36',
  'p49',
  'p58',
  'p57',
  'p47',
  'p38',
  'p53',
  'p54',
  'p44',
  'p59',
  'p23',
  'p6',
  'p385',
  'p388',
  'p3',
  'p11'],
 'verdiepingaanduiding': 't9'}

In [17]:
float((bbx.maxy-bbx.miny).iloc[0])

30.79500000004191

In [22]:
data['points']['p10']['position']

[1499.4282453727287, 3148.921159935223]

In [21]:
data['text'].keys()

dict_keys(['t2', 't3', 't4', 't5', 't6', 't7', 't8', 't9'])

In [12]:
kad_list

['VW_HVS00_N_1878',
 'VW_HVS00_N_1882',
 'VW_HVS00_N_2359',
 'VW_HVS00_N_2643',
 'VW_HVS00_N_2848',
 'VW_HVS00_N_3211',
 'VW_HVS00_N_3723',
 'VW_HVS00_N_4216',
 'VW_HVS00_N_555',
 'VW_HVS00_N_8954',
 'VW_HVS00_N_9252']